In [268]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [269]:
# This is basically the numerical representation of the audio file
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data,2,0)
#Read the df
df = pd.read_csv('primary_dataframe.csv')
y = df.diagnosis_Healthy
del df # Get this out of memory

In [270]:
# Load the data
data = np.load('arr_mfcc.npy')
data = np.moveaxis(data, 2, 0)

# Create a list to store the data
data_list = []

# Loop over each audio file
for i in range(len(data)):
    # Loop over each time step
    time_steps = []
    for j in range(1, len(data[i])):
        time_steps.append(data[i][j])
    
    # Append the data for this audio file to the list
    data_list.append(time_steps)

# Create the dataframe
df = pd.DataFrame({'data': data_list, 'label': y})

This code combines to get rid of all the 0s to make the dataset balanced

In [271]:
# Count the number of rows with label 0 and 1
num_0 = df['label'].value_counts()[0]
num_1 = df['label'].value_counts()[1]


# Calculate the number of rows to remove
num_to_remove = abs(num_0 - num_1)-100

# Remove rows with the minimum label until the number of rows for each label is equal
df_removed = df.drop(df[df['label'] == 0].sample(num_to_remove).index)

# Save removed rows to a separate DataFrame
df_removed_only = df.loc[df.index.difference(df_removed.index)]

#Replace back into df
df = df_removed
del df_removed

In [272]:
from sklearn.model_selection import train_test_split

# Extract the data and labels from the dataframe
X = df['data'].values
y = df['label'].values

# Split the data and labels into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [273]:
# Reshape X_train and X_test to have shape (num_samples, num_features)
X_train = np.array([np.array(x).reshape(-1) for x in X_train])
X_test = np.array([np.array(x).reshape(-1) for x in X_test])


In [274]:
from sklearn.svm import SVC

# Create an SVM model with a linear kernel
model = SVC(kernel='linear')

# Train the model on the training data
model.fit(X_train, y_train)

# Evaluate the model on the test data
acc = model.score(X_test, y_test)
print(f"Accuracy: {acc}")


Accuracy: 0.9117647058823529


In [275]:
from sklearn.metrics import classification_report

# Evaluate the model on the test data and print precision, recall, and F1-score
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.93      0.97      0.95        29
         1.0       0.75      0.60      0.67         5

    accuracy                           0.91        34
   macro avg       0.84      0.78      0.81        34
weighted avg       0.91      0.91      0.91        34



In [276]:
# Create a labelled confusion matrix
cm = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'])
print(cm)

Predicted  0.0  1.0
Actual             
0.0         28    1
1.0          2    3


In [277]:
#Testing w the biased data
X_testb = df_removed_only['data'].values
X_testb = np.array([np.array(x).reshape(-1) for x in X_testb])

y_testb = df_removed_only['label'].values
y_pred = model.predict(X_testb)

#Metrics
print(classification_report(y_testb, y_pred))
# Create a labelled confusion matrix
cm = pd.crosstab(y_testb, y_pred, rownames=['Actual'], colnames=['Predicted'])

              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98       750
         1.0       0.00      0.00      0.00         0

    accuracy                           0.96       750
   macro avg       0.50      0.48      0.49       750
weighted avg       1.00      0.96      0.98       750



/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/salimalkharsa/Desktop/ECEN360_Final/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

In [278]:
print(cm)

Predicted  0.0  1.0
Actual             
0.0        722   28
